In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data

import random, os, pathlib, time
from tqdm import tqdm
from sklearn import datasets

In [2]:
import nflib
from nflib.flows import SequentialFlow, NormalizingFlow, ActNorm, AffineConstantFlow
import nflib.coupling_flows as icf
import nflib.inn_flow as inn
import nflib.res_flow as irf

In [3]:
import dtnnlib as dtnn

In [4]:
device = torch.device("cuda:0")
# device = torch.device("cpu")

## MNIST dataset

In [5]:
import mylibrary.datasets as datasets

In [6]:
mnist = datasets.FashionMNIST()
# mnist.download_mnist()
# mnist.save_mnist()
train_data, train_label_, test_data, test_label_ = mnist.load()

train_data = train_data / 255.
test_data = test_data / 255.

# train_label = tnn.Logits.index_to_logit(train_label_)
train_size = len(train_label_)

In [7]:
## converting data to pytorch format
train_data = torch.Tensor(train_data)
test_data = torch.Tensor(test_data)
train_label = torch.LongTensor(train_label_)
test_label = torch.LongTensor(test_label_)

In [8]:
input_size = 784
output_size = 10

In [9]:
class MNIST_Dataset(data.Dataset):
    
    def __init__(self, data, label):
        self.data = data
        self.label = label
        
#         self.label = mask.type(torch.float32).reshape(-1,1)
        self._shuffle_data_()
        
    def __len__(self):
        return len(self.data)
    
    def _shuffle_data_(self):
        randidx = random.sample(range(len(self.data)), k=len(self.data))
        self.data = self.data[randidx]
        self.label = self.label[randidx]
    
    def __getitem__(self, idx):
        img, lbl = self.data[idx], self.label[idx]
        return img, lbl

In [10]:
train_dataset = MNIST_Dataset(train_data, train_label)
test_dataset = MNIST_Dataset(test_data, test_label)

In [11]:
learning_rate = 0.0003
batch_size = 50

In [12]:
train_loader = data.DataLoader(dataset=train_dataset, num_workers=4, batch_size=batch_size, shuffle=True)
test_loader = data.DataLoader(dataset=test_dataset, num_workers=4, batch_size=batch_size, shuffle=False)

In [254]:
class ParameterSelector(nn.Module):
    
    def __init__(self, input_dim, num_sets, parameter_shapes, inv_temp=1):
        super().__init__()
        
        self.input_dim = input_dim
        self.num_sets = num_sets
        self.parameter_shapes = parameter_shapes
#         self.inv_temp = nn.Parameter(torch.ones(1)*inv_temp)
        self.inv_temp = inv_temp
        
        self.dt = dtnn.DistanceTransformBase(input_dim, num_sets)
        
        self.parameter_list = []
        for ps in self.parameter_shapes:
            ps = (self.num_sets, *ps)
            if len(ps) > 1:
                param = torch.nn.init.xavier_uniform_(torch.empty(*ps), gain=1.0).reshape(self.num_sets, -1)
            else:
                param = torch.zeros(*ps)
#             print(ps, param)
            self.parameter_list.append(nn.Parameter(param))
        
        self.parameter_list = nn.ParameterList(self.parameter_list)

        ## class repeat sequentially
        self.cls_confidence = None
        
    def forward(self, x, hard=False):
        
        dists = self.dt(x)
        dists = dists/np.sqrt(self.input_dim) ### correction to make diagonal of unit square 1 in nD space
        if hard:
            x = torch.softmax(-dists*1e5, dim=1)
        else:
            x = torch.softmax(-dists*self.inv_temp, dim=1)
        
        self.cls_confidence = x
        
        ret_params = []
        for param, shape in zip(self.parameter_list, self.parameter_shapes):
#             print(x.shape, param.shape)
            p = x@param
#             print(p.shape)
            ret_params.append(p.reshape(-1, *shape))
        return ret_params

In [255]:
ps = ParameterSelector(10, 5, [(10,2), (2,)])

In [256]:
list(ps.parameters())

[Parameter containing:
 tensor([[-0.1313,  0.0607, -0.2348, -0.7330, -0.1623, -0.5914,  0.2529, -0.0696,
          -0.5902, -0.5673],
         [-0.5802, -0.4504, -0.1569, -0.1745,  0.1463,  0.2057,  0.0067, -0.3910,
          -0.1000, -0.0230],
         [ 0.0756,  0.2478,  0.1728, -0.3095,  0.1257, -0.1759,  0.2901,  0.1017,
           0.1039,  0.0228],
         [-1.0776,  0.1633, -0.1290,  0.0582,  0.3161,  0.2448,  0.4085,  0.4782,
          -0.4364, -0.3307],
         [ 0.2917,  0.6311,  0.0644, -0.1773, -0.5003, -0.1194, -0.5465,  0.2802,
           0.3461,  0.4844]], requires_grad=True),
 Parameter containing:
 tensor([[-0.1443, -0.0646, -0.0770, -0.2922,  0.3557,  0.1664, -0.1609, -0.3694,
           0.2793,  0.1567,  0.2841, -0.2629,  0.0418, -0.1899,  0.0156, -0.0733,
           0.2299, -0.2937, -0.3215, -0.2278],
         [ 0.2329,  0.2523, -0.0409,  0.4462, -0.1542,  0.4059, -0.0168,  0.1805,
           0.2208, -0.2575,  0.3665,  0.2329, -0.3225,  0.1412, -0.2589, -0.3646,
  

In [257]:
x = torch.randn(3, 10)
params = ps(x)

In [258]:
params[1].shape

torch.Size([3, 2])

In [259]:
def psLinear(x, weights, bias):
    return torch.bmm(x.unsqueeze(1), weights).squeeze(1) + bias

In [260]:
psLinear(x, params[0], params[1])

tensor([[-1.1387,  0.0128],
        [-0.1935, -0.3205],
        [-0.4597, -0.4354]], grad_fn=<AddBackward0>)

## Now the model

In [261]:
# actf = irf.Swish
# flows = [
#     ActNorm(784),
#     irf.ResidualFlow(784, [784], activation=actf),
#     ActNorm(784),
#     irf.ResidualFlow(784, [784], activation=actf),
#     ActNorm(784),
#         ]

# model = SequentialFlow(flows)

In [262]:
model = nn.Sequential(nn.Linear(784, 784, bias=False),
#                       nn.BatchNorm1d(784),
#                       nn.SELU(),
#                       nn.Linear(784, 784, bias=False),
#                       nn.BatchNorm1d(784),
#                       nn.SELU(),
                     )

In [263]:
model.to(device)

Sequential(
  (0): Linear(in_features=784, out_features=784, bias=False)
)

In [277]:
ps0 = ParameterSelector(784, 20, [(784, 10), (10,)])
ps0 = ps0.to(device)

## Model Train

In [278]:
# criterion = nn.NLLLoss()
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(list(model.parameters())+list(ps0.parameters()),
                       lr=0.0003) # todo tune WD
# optimizer = optim.SGD(model.parameters(), lr=0.1)

print("number of params: ", sum(p.numel() for p in model.parameters()))

number of params:  614656


In [279]:
# for p in model.parameters():
#     print(torch.isnan(p).type(torch.float32).sum())

In [280]:
xx = iter(test_loader).next()[0]
xx.shape

torch.Size([50, 784])

In [281]:
model(xx.to(device))

tensor([[ 1.1920,  0.6853,  0.2957,  ...,  1.3573,  0.1067,  1.1073],
        [ 0.6517, -0.2408, -0.8800,  ..., -1.2877,  1.6682,  0.4547],
        [ 0.4065,  0.1481,  0.8354,  ..., -1.0897,  2.0784, -1.6912],
        ...,
        [-0.8900,  1.7951, -1.8275,  ..., -0.0817, -1.7312,  2.0230],
        [ 1.5230,  0.3577, -0.6214,  ...,  2.1484, -1.8978,  1.8731],
        [ 0.0172,  0.0642, -0.8129,  ..., -0.6853,  0.0893,  0.2986]],
       device='cuda:0', grad_fn=<MmBackward>)

In [282]:
losses = []
train_accs = []
test_accs = []
EPOCHS = 99

index = 0
for epoch in range(EPOCHS):
    if (epoch+1)%10 == 0:
        ps0.inv_temp *= np.e
    
    train_acc = 0
    train_count = 0
    for xx, yy in tqdm(train_loader):
        xx, yy = xx.to(device), yy.to(device)

        yout = model(xx)
        params = ps0(yout)
        yout = psLinear(yout, params[0], params[1])
        
        loss = criterion(yout, yy)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(float(loss))

        outputs = torch.argmax(yout, dim=1).data.cpu().numpy()
        correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
        train_acc += correct
        train_count += len(outputs)

    train_accs.append(float(train_acc)/train_count*100)
    train_acc = 0
    train_count = 0

    print(f'Epoch: {epoch}:{index},  Loss:{float(loss)}')
    test_count = 0
    test_acc = 0
    for xx, yy in tqdm(test_loader):
        xx, yy = xx.to(device), yy.to(device)
        with torch.no_grad():
            yout = model(xx)
            params = ps0(yout)
            yout = psLinear(yout, params[0], params[1])
        outputs = torch.argmax(yout, dim=1).data.cpu().numpy()
        correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
        test_acc += correct
        test_count += len(xx)
    test_accs.append(float(test_acc)/test_count*100)
    print(f'Train Acc:{train_accs[-1]:.2f}%, Test Acc:{test_accs[-1]:.2f}%')
    print()

### after each class index is finished training
print(f'\t-> Train Acc {max(train_accs)} ; Test Acc {max(test_accs)}')

  0%|          | 0/200 [00:00<?, ?it/s]

Epoch: 0:0,  Loss:0.47675830125808716


  0%|          | 0/1200 [00:00<?, ?it/s]

Train Acc:83.74%, Test Acc:82.21%



  0%|          | 0/200 [00:00<?, ?it/s]

...

Train Acc:92.12%, Test Acc:86.33%



  0%|          | 0/200 [00:00<?, ?it/s]

Epoch: 98:0,  Loss:0.21769079566001892


100%|██████████| 200/200 [00:00<00:00, 600.49it/s]

Train Acc:91.23%, Test Acc:85.58%

	-> Train Acc 95.94833333333334 ; Test Acc 87.98


In [ ]:
##### when trained for 30 epochs
## sets = 10: -> -> Train Acc 95.145 ; Test Acc 89.25
## sets = 2: -> -> Train Acc 95.1367 ; Test Acc 89.26

In [283]:
ps0.inv_temp

8103.0839275753815

### Hard test accuracy with count per classifier

In [284]:
test_count = 0
test_acc = 0
set_count = torch.zeros(ps0.num_sets).to(device)
for xx, yy in tqdm(test_loader):
    xx, yy = xx.to(device), yy.to(device)
    with torch.no_grad():
        yout = model(xx)
        params = ps0(yout, hard=True)
        yout = psLinear(yout, params[0], params[1])
        set_indx, count = torch.unique(torch.argmax(ps0.cls_confidence, dim=1), return_counts=True) 
        set_count[set_indx] += count
    outputs = torch.argmax(yout, dim=1).data.cpu().numpy()
    correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
    test_acc += correct
    test_count += len(xx)

print(f'Hard Test Acc:{float(test_acc)/test_count*100:.2f}%')
print(set_count.type(torch.long).tolist())

100%|██████████| 200/200 [00:00<00:00, 561.21it/s]

Hard Test Acc:85.54%
[149, 190, 30, 223, 240, 172, 118, 1304, 239, 85, 303, 852, 749, 638, 1396, 275, 1574, 774, 158, 531]


In [285]:
### 100 epochs
# 10 sets == Hard Test Acc:85.05%
# 20 sets == Hard Test Acc:85.54%

### Hard train accuracy with count per classifier

In [286]:
test_count = 0
test_acc = 0
set_count = torch.zeros(ps0.num_sets).to(device)
for xx, yy in tqdm(train_loader):
    xx, yy = xx.to(device), yy.to(device)
    with torch.no_grad():
        yout = model(xx)
        params = ps0(yout, hard=True)
        yout = psLinear(yout, params[0], params[1])
        set_indx, count = torch.unique(torch.argmax(ps0.cls_confidence, dim=1), return_counts=True) 
        set_count[set_indx] += count
    outputs = torch.argmax(yout, dim=1).data.cpu().numpy()
    correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
    test_acc += correct
    test_count += len(xx)

print(f'Hard Train Acc:{float(test_acc)/test_count*100:.2f}%')
print(set_count.type(torch.long).tolist())

100%|██████████| 1200/1200 [00:01<00:00, 809.02it/s]

Hard Train Acc:92.04%
[904, 1165, 192, 1488, 1451, 1100, 690, 7644, 1281, 479, 1861, 4823, 4466, 4009, 8821, 1582, 9592, 4464, 963, 3025]


In [201]:
#### Classifiers that enclose any data
torch.count_nonzero(set_count)

tensor(2, device='cuda:0')

In [50]:
#### classifier with class representation
torch.argmax(classifier.cls_weight, dim=1)

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3,
        4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7,
        8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1,
        2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5,
        6, 7, 8, 9], device='cuda:0')

In [51]:
# The class labels are same as that of initialized
# tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3,
#         4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7,
#         8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1,
#         2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5,
#         6, 7, 8, 9], device='cuda:0')

In [52]:
classifier.cls_weight

Parameter containing:
tensor([[6.2399e-01, 4.8636e-06, 9.7206e-05, 8.1379e-05, 4.9807e-05, 5.0686e-06,
         3.7574e-01, 9.6449e-06, 7.1354e-06, 8.5779e-06],
        [5.1824e-05, 9.9977e-01, 1.3589e-05, 2.3619e-05, 2.1795e-05, 2.3727e-05,
         2.6346e-05, 1.3267e-05, 3.7349e-05, 1.6489e-05],
        [1.8427e-04, 7.4690e-05, 6.3821e-01, 3.0832e-05, 1.6561e-01, 5.7689e-05,
         1.9566e-01, 4.7485e-05, 7.2066e-05, 4.7959e-05],
        [1.9669e-04, 1.7147e-04, 1.3287e-04, 9.9875e-01, 1.9478e-04, 1.0700e-04,
         7.7105e-05, 1.0586e-04, 1.7403e-04, 9.5043e-05],
        [9.4674e-05, 3.5800e-05, 2.7890e-01, 4.9266e-05, 6.5840e-01, 3.3135e-05,
         6.2412e-02, 2.1661e-05, 2.8074e-05, 2.7890e-05],
        [3.1471e-04, 3.0296e-04, 2.9542e-04, 2.8826e-04, 3.0731e-04, 9.9767e-01,
         2.7568e-04, 1.4418e-04, 2.7908e-04, 1.2719e-04],
        [3.7181e-05, 2.0084e-04, 2.1955e-01, 1.2258e-04, 2.1433e-01, 1.9574e-04,
         5.6498e-01, 2.0206e-04, 1.8711e-04, 2.0165e-04],
     

In [53]:
# torch.unique(torch.argmax(classifier.cls_confidence, dim=1), return_counts=True)

In [54]:
classifier.inv_temp

Parameter containing:
tensor([3.0942], device='cuda:0', requires_grad=True)

In [55]:
### example output per classifier
yout[5]

tensor([1.9399e-04, 1.7627e-04, 1.3068e-04, 9.9873e-01, 2.0075e-04, 1.0751e-04,
        7.0551e-05, 1.0562e-04, 1.8927e-04, 9.5756e-05], device='cuda:0')

In [134]:
asdfsdf ## to break the code

NameError: name 'asdfsdf' is not defined

### analyze per classifier accuracy

In [158]:
test_count = 0
test_acc = 0
set_count = torch.zeros(classifier.num_sets).to(device)
set_acc = torch.zeros(classifier.num_sets).to(device)
for xx, yy in tqdm(train_loader):
    xx, yy = xx.to(device), yy.to(device)
    with torch.no_grad():
        yout = classifier(model(xx), hard=True)
        
    cls_indx = torch.argmax(classifier.cls_confidence, dim=1)
    set_indx, count = torch.unique(cls_indx, return_counts=True) 
    set_count[set_indx] += count
    
    outputs = torch.argmax(yout, dim=1).data.cpu().numpy()
    correct = (outputs == yy.data.cpu().numpy()).astype(float)
    
    ### class_index has 100 possible values
    for i, c in enumerate(correct):
        set_acc[cls_indx[i]] += c
    
#     print(set_acc.sum(), set_count.sum())
#     break
    test_acc += correct.sum()
    test_count += len(xx)

print(f'Hard Train Acc:{float(test_acc)/test_count*100:.2f}%')
print(set_count.type(torch.long).tolist())

100%|██████████| 1200/1200 [00:19<00:00, 60.60it/s]

Hard Train Acc:94.22%
[4, 70, 5920, 0, 19, 0, 12, 3, 0, 2, 5, 0, 0, 6080, 0, 0, 0, 5984, 50, 0, 1, 7, 0, 8, 1, 1, 13, 0, 0, 14, 36, 1, 0, 1, 0, 971, 1876, 5, 7, 1, 71, 0, 12, 32, 0, 0, 5819, 5, 11, 0, 162, 0, 77, 6092, 37, 6243, 5, 343, 542, 205, 7, 1, 13, 16, 3, 5572, 3846, 7, 3197, 12, 0, 1, 1, 1, 0, 0, 6, 132, 4, 0, 32, 80, 25, 11, 0, 19, 37, 1, 0, 1, 0, 188, 0, 0, 48, 0, 3, 5971, 17, 0]


In [161]:
set_acc/set_count

tensor([1.0000, 0.5714, 0.9965,    nan, 0.5789,    nan, 1.0000, 0.6667,    nan,
        0.0000, 1.0000,    nan,    nan, 0.9704,    nan,    nan,    nan, 0.9883,
        0.5200,    nan, 1.0000, 0.8571,    nan, 1.0000, 1.0000, 0.0000, 0.1538,
           nan,    nan, 0.5714, 1.0000, 1.0000,    nan, 1.0000,    nan, 1.0000,
        0.7820, 0.2000, 0.2857, 0.0000, 1.0000,    nan, 1.0000, 1.0000,    nan,
           nan, 0.9443, 0.0000, 0.5455,    nan, 0.9877,    nan, 0.9870, 0.9703,
        0.2703, 0.9031, 0.6000, 0.9883, 1.0000, 0.7171, 0.4286, 0.0000, 1.0000,
        0.5000, 1.0000, 0.9248, 0.9280, 0.7143, 0.9984, 0.3333,    nan, 1.0000,
        0.0000, 1.0000,    nan,    nan, 0.6667, 0.5076, 0.2500,    nan, 0.5625,
        0.9750, 0.2800, 0.1818,    nan, 0.8947, 1.0000, 0.0000,    nan, 0.0000,
           nan, 0.9840,    nan,    nan, 0.9167,    nan, 0.3333, 0.8970, 0.2941,
           nan], device='cuda:0')

In [162]:
print(f"Index\tNumData\tClass\tAccuracy")
for i, (cnt, acc, cls) in enumerate(zip(set_count.type(torch.long).tolist(),
                                   (set_acc/set_count).tolist(),
                                   torch.argmax(classifier.cls_weight, dim=1).tolist())):
    if cnt == 0: continue
    print(f"{i}\t {cnt}\t {cls}\t {acc*100:.2f}%")

Index	NumData	Class	Accuracy
0	 4	 5	 100.00%
1	 70	 3	 57.14%
2	 5920	 1	 99.65%
4	 19	 4	 57.89%
6	 12	 5	 100.00%
7	 3	 4	 66.67%
9	 2	 6	 0.00%
10	 5	 6	 100.00%
13	 6080	 7	 97.04%
17	 5984	 8	 98.83%
18	 50	 3	 52.00%
20	 1	 4	 100.00%
21	 7	 5	 85.71%
23	 8	 5	 100.00%
24	 1	 6	 100.00%
25	 1	 7	 0.00%
26	 13	 6	 15.38%
29	 14	 6	 57.14%
30	 36	 5	 100.00%
31	 1	 4	 100.00%
33	 1	 4	 100.00%
35	 971	 5	 100.00%
36	 1876	 6	 78.20%
37	 5	 4	 20.00%
38	 7	 6	 28.57%
39	 1	 7	 0.00%
40	 71	 5	 100.00%
42	 12	 5	 100.00%
43	 32	 5	 100.00%
46	 5819	 3	 94.43%
47	 5	 8	 0.00%
48	 11	 4	 54.55%
50	 162	 5	 98.77%
52	 77	 5	 98.70%
53	 6092	 9	 97.03%
54	 37	 6	 27.03%
55	 6243	 0	 90.31%
56	 5	 4	 60.00%
57	 343	 5	 98.83%
58	 542	 5	 100.00%
59	 205	 3	 71.71%
60	 7	 6	 42.86%
61	 1	 4	 0.00%
62	 13	 5	 100.00%
63	 16	 6	 50.00%
64	 3	 5	 100.00%
65	 5572	 2	 92.48%
66	 3846	 6	 92.80%
67	 7	 4	 71.43%
68	 3197	 5	 99.84%
69	 12	 6	 33.33%
71	 1	 6	 100.00%
72	 1	 5	 0.00%
73	 1	 6	 